Text Processing: Clean and preprocess the extracted HTML data:

Remove HTML tags and scripts.
Normalize text (lowercasing, removing punctuation).
Tokenization and stop-word removal can be beneficial.


Feature Extraction: Identify and extract relevant features that could help classify a domain as an eCommerce store:

Keywords and phrases.
HTML structure elements (e.g., presence of <product>, <price>, <cart> tags).
Meta tags relevant to eCommerce.


LLM Utilization:

Use large language models (like GPT-3, BERT) to classify extracted text or assist in feature selection. Fine-tuning these models on your labeled data could yield excellent results.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import requests
import warnings
from urllib.parse import urljoin
import re
from googletrans import Translator
import nltk
import time
import requests
from urllib.parse import urlencode
from deep_translator import GoogleTranslator
import pandas as pd
import unicodedata
import spacy
import numpy as np
import emoji



In [2]:
data_1=pd.read_csv('noshop.csv').iloc[:1000,:]

In [ ]:
# # Phrases to search for
# phrases_to_search = [
#     "Terms and conditions",
#     "Privacy policy",
#     "Return and exchange policy",
#     "Place a return or exchange request",
#     "Track your order",
#     "Add to cart",
#     "add to cart",
#     "select product",
#     "Online shop",
#     "Shipping & Payment",
#     "Exchanges & Refunds",
#     "How to Shop",
#     "Order Status",
#     "Shipping Method",
#     "Trackin order",
#     "User Guide ",
#     "online e-commerce",
#     "Quick buy",
#     "Easy Return Guidance",
#     "Shipping Details",
#     "Online shop",
#     "Delivery",
#     "RETURNS AND EXCHANGE",  
#     "Payment methods",
#     "Returns and refunds",
#     "Shipping Information & Costs",
#     "Shopping basket",
#     "Loyalte program",
#     "Gift Cards",
#     "Shipping and payment",
#     "Returns / Product exchange",
#     "Payment", 
#     "Shipping & Delivery",
#     "Exchanges & Returns",
#     "Shipping and Exchange Policy",
#     "Shopping basket",
#     "Refund policy",
#     "All about shopping",
#     "Basket",
#     "Shopping cart",
#     "Return procedure",
#     "Payment methods",
#     "Guarantees and returns",
#     "Return/Complaint Instructions",
#     "About Click & Collect", # "https://www.bike-online.jp/hpgen/HPB/entries/43.html"
#     "E-shop",
#     "order status "
#     ]

In [3]:
# # Suppress SSL warnings
# # warnings.filterwarnings("ignore", category=requests.packages.urllib3.exceptions.InsecureRequestWarning)
# warnings.simplefilter(action='ignore')
# # Function to fetch HTML content from the domain
# def fetch_html(domain):
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#     }
#     try:
#         response = requests.get(f'https://{domain}', headers=headers, timeout=10, verify=False)
#         if response.status_code == 200:
#             return response.text
#         else:
#             return ""
#     except requests.RequestException:
#         return ""

# # Function to extract and clean text from the HTML
# def extract_text_from_domain(domain):
#     # Fetch HTML content using fetch_html
#     html = fetch_html(domain)
#     if not html:
#         return "Error fetching the webpage."

#     try:
#         soup = BeautifulSoup(html, "html.parser")

#         # Extract text from all elements and clean it
#         text = soup.get_text(separator=" ")
#         cleaned_text = re.sub(r"\s+", " ", text).strip()

#         return cleaned_text
#     except Exception as e:
#         return f"Error extracting text: {e}"

# # Function to search for specific phrases
# def search_phrases(html_content, phrases):
#     if not isinstance(html_content, str):
#         return ''
#     found_phrases = []
#     for phrase in phrases:
#         if phrase.lower() in html_content.lower():
#             found_phrases.append(phrase)
#     return ', '.join(found_phrases)

# # Function to translate text
# def translate_text(text, src_language):
#     translator = Translator()
#     translation = translator.translate(text, src=src_language, dest="en")

# # Function to detect the HTML language
# def detect_language(html_content):
#     if not html_content:
#         return None
    
#     # Use regex to find the lang attribute directly
#     lang_match = re.search(r'<html[^>]*\blang=["\']?([a-zA-Z-]+)', html_content, re.IGNORECASE)
#     if lang_match:
#         detected_lang = lang_match.group(1).split('-')[0]  # Extract primary language
#         return detected_lang

#     # Fallback to BeautifulSoup parsing
#     soup = BeautifulSoup(html_content, 'lxml')
#     html_tag = soup.html
#     if html_tag and html_tag.get('lang'):
#         detected_lang = html_tag.get('lang').split('-')[0]
#         return detected_lang
#     return ""

# # Apply functions to DataFrame
# data_1['HTML'] = data_1['domain'].apply(fetch_html)
# data_1['Text'] = data_1['domain'].apply(extract_text_from_domain)
# data_1['found_phrases'] = data_1['HTML'].apply(lambda html: search_phrases(html, phrases_to_search))
# data_1['language'] = data_1['HTML'].apply(detect_language)
# data_1.head(4)

Cleaning functions:

-  Cleaning:

In [3]:
data_1 = data_1[data_1['HTML'].apply(lambda x: x not in [[], ""])]
data_1 = data_1[data_1["Text"].apply(lambda x: not isinstance(x, float))]


In [4]:
data_1.shape

(986, 5)

In [5]:
dict_symbols_to_change = {
    "\u2714": ' ',  # ✔
    "\u2605": ' ',  # ★
    "\u00A9": ' ',  # ©
    "\u00AE": ' ',  # ®
    "\u2022": ' ',  # •
    "\u2798": ', ',  # ➨
    "\u27BD": ', ',  # ➽
    "\u2661": ', ',  # ♡
    "\u266A": ', ',  # ♪
    "\u2026": ', ',  # …
    "\u25CF": ', ',  # ●
    "\u2122": ' ',  # ™
    "\u2713": ' ',  # ✓
    "\u2600": ' ',  # ☀ (Stars and Sun)
    "\u2665": ' ',  # ♥ 
    "\U000025BA": ' ', # ►
}

In [6]:
def replace_symbols(text):
    """
    Replace some symbols in text as defined in lookup.dict_symbols_to_change.
    """
    for key, value in dict_symbols_to_change.items():
        text = text.replace(key, value)
    return text


In [7]:
def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')  # Replace emojis with empty string


In [8]:
data_1["Text"]=data_1["Text"].apply(replace_symbols)
data_1["Text"]=data_1["Text"].apply(remove_emojis)

In [9]:
data_1["len"]=data_1["Text"].apply(len)
data_1.sort_values("len",ascending=False).head(5)


,domain,HTML,Text,found_phrases,language,len
695,arena.pl,"<!DOCTYPE html><html lang=""pl""><head><link rel...",Arena.pl Platforma zakupowa Bezpieczne zak...,"Delivery, Payment, Basket, E-shop",pl,1079239
614,anzlitlovers.com,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",ANZ LitLovers LitBlog | For lovers of Australi...,"Delivery, Payment",en,241540
701,arikair.com,"<!DOCTYPE html>\n<html class=""load-full-screen...",Fly Arik Air - West-Africa's leading airline o...,"Terms and conditions, Delivery, Payment",NaN,102234
566,android8o.com,"<!DOCTYPE html>\r\n<html lang=""ru"">\r\n<head>\...",Прошивки Android Oreo 8 для смартфонов и планш...,Basket,ru,98454
628,apiajapan.com,"<!DOCTYPE html>\n\t<html lang=""ja"" prefix=""og:...",APIA -ã¢ãã¢- APIA ABOUT APIA ã¢ãã¢ã«ã...,NaN,ja,96398


In [10]:
data_1 = data_1[(data_1["len"] >= 1000) & (data_1["len"] <= 100000)]

In [11]:
data_1=data_1.drop(columns=["HTML","len"])
data_1.shape

(755, 4)

In [12]:
import html
import re
import unicodedata

def normalize_text(text):
    if not isinstance(text, str):  # Ensure input is a string
        return text
    
    # Decode HTML entities
    text = html.unescape(text)
    
    # Normalize Unicode (preserve foreign letters, remove accents)
    text = ''.join(c for c in unicodedata.normalize('NFKC', text) if unicodedata.category(c) != 'Mn')

    # Remove control characters (except line breaks)
    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', text)  

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

# Apply normalization function
data_1['normalized_text'] = data_1['Text'].astype(str).apply(normalize_text)

# Display the result
data_1.head(3)


,domain,Text,found_phrases,language,normalized_text
0,0101shop.com,å åç ´è§£æ¸¸æ-2020å åç ´è§£æ¸¸æä¸è½...,NaN,NaN,å åç è§£æ æ-2020å åç è§£æ æä è1⁄21⁄...
6,1001cartes.com,"Faire-part de naissance, mariage, baptême - vo...","Add to cart, add to cart, Shipping Method, Pay...",fr,"Faire-part de naissance, mariage, baptême - vo..."
7,1001cuponesdedescuento.cl,Cupones de descuento en Chile hasta 80% OFF en...,"Delivery, E-shop",es,Cupones de descuento en Chile hasta 80% OFF en...


In [ ]:
import re

def remove_consecutive_duplicates(text):
    return re.sub(r'\b(\w+)(\s+\1)+\b', r'\1', text)


,domain,Text,found_phrases,language,normalized_text
0,0101shop.com,å åç ´è§£æ¸¸æ-2020å åç ´è§£æ¸¸æä¸è½...,NaN,NaN,å åç è§£æ æ-2020å åç è§£æ æä è1⁄21⁄...
6,1001cartes.com,"Faire-part de naissance, mariage, baptême - vo...","Add to cart, add to cart, Shipping Method, Pay...",fr,"Faire-part de naissance, mariage, baptême - vo..."
7,1001cuponesdedescuento.cl,Cupones de descuento en Chile hasta 80% OFF en...,"Delivery, E-shop",es,Cupones de descuento en Chile hasta 80% OFF en...
8,100comments.com,"Product Reviews, Free Samples, New Products @ ...",Privacy policy,en,"Product Reviews, Free Samples, New Products @ ..."


In [16]:
data_1['normalized_text']=data_1['normalized_text'].apply(remove_consecutive_duplicates)
data_1

,domain,Text,found_phrases,language,normalized_text
0,0101shop.com,å åç ´è§£æ¸¸æ-2020å åç ´è§£æ¸¸æä¸è½...,NaN,NaN,å åç è§£æ-2020å åç è§£æä è1⁄21⁄2-01...
6,1001cartes.com,"Faire-part de naissance, mariage, baptême - vo...","Add to cart, add to cart, Shipping Method, Pay...",fr,"Faire-part de naissance, mariage, baptême - vo..."
7,1001cuponesdedescuento.cl,Cupones de descuento en Chile hasta 80% OFF en...,"Delivery, E-shop",es,Cupones de descuento en Chile hasta 80% OFF en...
8,100comments.com,"Product Reviews, Free Samples, New Products @ ...",Privacy policy,en,"Product Reviews, Free Samples, New Products @ ..."
9,100sp.ru,100sp — интернет-магазин. Товары по выгодным ц...,Delivery,ru,100sp — интернет-магазин. Товары по выгодным ц...
...,...,...,...,...,...
992,baermate.com,"BAER-MATE: Sabor, Saúde e Energia, 100% Natura...",Payment,pt,"BAER-MATE: Sabor, Saúde e Energia, 100% Natura..."
993,bafa.de,BAFA - Startseite Springe direkt zu: Inhalt Ha...,NaN,de,BAFA - Startseite Springe direkt zu: Inhalt Ha...
995,baileylineroad.com,Baileylineroad Skip to content COURSES Expand ...,NaN,NaN,Baileylineroad Skip to content COURSES Expand ...
998,bakeryandsnacks.com,"Bakery and snacks, cereal, cakes and pastries ...","Delivery, Shopping cart, E-shop",en,"Bakery and snacks, cereal, cakes and pastries ..."


In [ ]:
# data_1["lang_len"]=data_1["language"].astype(str).apply(len)
# data_1 = data_1[data_1["lang_len"] == 2]


In [17]:
data_1.shape

(755, 5)

In [21]:

def decode_last(text):
    encoded_text = text

    # Using UTF-8 encoding and ignoring errors
    decoded_text = encoded_text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
    return decoded_text


In [22]:
data_1["Decode"]=data_1["normalized_text"].apply(decode_last)


In [24]:
data_1.to_excel("noshop_decoded.xlsx")

___________________


# Text Translation pipeline


In [21]:
# import spacy
# import pandas as pd

# # Load the model once globally
# nlp = spacy.load("xx_ent_wiki_sm")
# nlp.add_pipe('sentencizer')

# def Tokenizer(text):
#     """Tokenize a text into sentences using spaCy."""
#     if pd.isna(text) or not isinstance(text, str):  # Handle NaN or non-string values
#         return []
    
#     doc = nlp(text)
#     return [sent.text.strip() for sent in doc.sents]  # Return a list of sentences

# # Apply the function to the 'text' column
# data_1['tokenized_sentences'] = data_1['normalized_text'].apply(Tokenizer)

# data_1.head(4)



In [26]:
from langdetect import detect

def detect_language(text):
    return detect(text)


In [28]:
data_1["detected_language"]=data_1["normalized_text"].apply(detect_language)

In [29]:
sum_mismatches = 0
for i in range(len(data_1)):  
    if data_1.iloc[i]["language"] != data_1.iloc[i]["detected_language"]:  
        sum_mismatches += 1 
print(sum_mismatches)


140


______

Failed translation model:

In [25]:
lang_map = {
    "af": "afr_Latn",
    "ak": "aka_Latn",
    "am": "amh_Ethi",
    "ar": "arb_Arab",
    "as": "asm_Beng",
    "ay": "ayr_Latn",
    "az": "azj_Latn",
    "bm": "bam_Latn",
    "be": "bel_Cyrl",
    "bn": "ben_Beng",
    "bho": "bho_Deva",
    "bs": "bos_Latn",
    "bg": "bul_Cyrl",
    "ca": "cat_Latn",
    "ceb": "ceb_Latn",
    "cs": "ces_Latn",
    "ckb": "ckb_Arab",
    "tt": "crh_Latn",
    "cy": "cym_Latn",
    "da": "dan_Latn",
    "de": "deu_Latn",
    "el": "ell_Grek",
    "en": "eng_Latn",
    "eo": "epo_Latn",
    "et": "est_Latn",
    "eu": "eus_Latn",
    "ee": "ewe_Latn",
    "fa": "pes_Arab",
    "fi": "fin_Latn",
    "fr": "fra_Latn",
    "gd": "gla_Latn",
    "ga": "gle_Latn",
    "gl": "glg_Latn",
    "gn": "grn_Latn",
    "gu": "guj_Gujr",
    "ht": "hat_Latn",
    "ha": "hau_Latn",
    "he": "heb_Hebr",
    "hi": "hin_Deva",
    "hr": "hrv_Latn",
    "hu": "hun_Latn",
    "hy": "hye_Armn",
    "nl": "nld_Latn",
    "ig": "ibo_Latn",
    "ilo": "ilo_Latn",
    "id": "ind_Latn",
    "is": "isl_Latn",
    "it": "ita_Latn",
    "jv": "jav_Latn",
    "ja": "jpn_Jpan",
    "kn": "kan_Knda",
    "ka": "kat_Geor",
    "kk": "kaz_Cyrl",
    "km": "khm_Khmr",
    "rw": "kin_Latn",
    "ko": "kor_Hang",
    "ku": "kmr_Latn",
    "lo": "lao_Laoo",
    "lv": "lvs_Latn",
    "ln": "lin_Latn",
    "lt": "lit_Latn",
    "lb": "ltz_Latn",
    "lg": "lug_Latn",
    "lus": "lus_Latn",
    "mai": "mai_Deva",
    "ml": "mal_Mlym",
    "mr": "mar_Deva",
    "mk": "mkd_Cyrl",
    "mg": "plt_Latn",
    "mt": "mlt_Latn",
    "mni-Mtei": "mni_Beng",
    "mni": "mni_Beng",
    "mn": "khk_Cyrl",
    "mi": "mri_Latn",
    "ms": "zsm_Latn",
    "my": "mya_Mymr",
    "no": "nno_Latn",
    "ne": "npi_Deva",
    "ny": "nya_Latn",
    "om": "gaz_Latn",
    "or": "ory_Orya",
    "pl": "pol_Latn",
    "pt": "por_Latn",
    "ps": "pbt_Arab",
    "qu": "quy_Latn",
    "ro": "ron_Latn",
    "ru": "rus_Cyrl",
    "sa": "san_Deva",
    "si": "sin_Sinh",
    "sk": "slk_Latn",
    "sl": "slv_Latn",
    "sm": "smo_Latn",
    "sn": "sna_Latn",
    "sd": "snd_Arab",
    "so": "som_Latn",
    "es": "spa_Latn",
    "sq": "als_Latn",
    "sr": "srp_Cyrl",
    "su": "sun_Latn",
    "sv": "swe_Latn",
    "sw": "swh_Latn",
    "ta": "tam_Taml",
    "te": "tel_Telu",
    "tg": "tgk_Cyrl",
    "tl": "tgl_Latn",
    "th": "tha_Thai",
    "ti": "tir_Ethi",
    "ts": "tso_Latn",
    "tk": "tuk_Latn",
    "tr": "tur_Latn",
    "ug": "uig_Arab",
    "uk": "ukr_Cyrl",
    "ur": "urd_Arab",
    "uz": "uzn_Latn",
    "vi": "vie_Latn",
    "xh": "xho_Latn",
    "yi": "ydd_Hebr",
    "yo": "yor_Latn",
    "zh-CN": "zho_Hans",
    "zh": "zho_Hans",
    "zh-TW": "zho_Hant",
    "zu": "zul_Latn",
    "pa": "pan_Guru"
}


In [26]:
# import torch
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# import re

# # Load NLLB model and tokenizer
# model_name = "facebook/nllb-200-distilled-600M"
# device = "cuda" if torch.cuda.is_available() else "cpu"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# def chunk_text(text, chunk_size=1000):
#     """Splits text into chunks of specified size, preserving sentences where possible."""
#     sentences = re.split(r'(?<=[.!?]) +', text)  # Split while keeping punctuation-based structure
#     chunks = []
#     current_chunk = []
#     current_length = 0
    
#     for sentence in sentences:
#         if current_length + len(sentence) + 1 > chunk_size:
#             chunks.append(" ".join(current_chunk))
#             current_chunk = []
#             current_length = 0
#         current_chunk.append(sentence)
#         current_length += len(sentence) + 1
    
#     if current_chunk:
#         chunks.append(" ".join(current_chunk))
    
#     return chunks

# def translate_chunks(text, src_lang, batch_size=4):
#     """Translates text efficiently by processing in batches, preserving non-text elements."""
#     if not text.strip():
#         return "[Empty content]"
    
#     if src_lang == "en":
#         return text  # Skip translation if language is English
    
#     src_lang_nllb = lang_map.get(src_lang, None)
#     if not src_lang_nllb:
#         return "[Unsupported language]"
    
#     chunks = chunk_text(text)
#     translated_chunks = []
    
#     for i in range(0, len(chunks), batch_size):
#         batch = chunks[i:i+batch_size]
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
#         translated_tokens = model.generate(
#             **inputs,
#             max_length=500,  # Increased limit for longer outputs
#             num_beams=2,  # More beams for better quality
#             forced_bos_token_id=tokenizer.convert_tokens_to_ids(["eng_Latn"])[0]
#         )
#         translations = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]
#         translated_chunks.extend(translations)
    
#     return " ".join(translated_chunks)

# # Apply translation with batch processing, skipping English rows
# data_1["translated_text"] = data_1.apply(
#     lambda row: row["normalized_text"] if row["language"] == "en" else translate_chunks(row["normalized_text"], row["language"]), axis=1
# )

# # Display final DataFrame
# data_1.head(4)


Successful transaltion model:

In [27]:
## Version I

# from deep_translator import GoogleTranslator
# import pandas as pd
# import ast  # To safely convert string representation of lists

# # Function to split long texts into chunks
# def chunk_text(text, chunk_size=2000):
#     return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

# # Function to translate tokenized sentences
# def translate_sentences(tokenized_list, src_lang="auto", target_lang="en"):
#     if isinstance(tokenized_list, list):
#         text = " ".join(tokenized_list)  # Convert list to string
#     else:
#         text = str(tokenized_list)  # Ensure it's a string
    
#     # Split text into smaller chunks
#     chunks = chunk_text(text)
    
#     # Translate each chunk separately
#     translated_chunks = [
#         GoogleTranslator(source=src_lang, target=target_lang).translate(chunk) for chunk in chunks
#     ]
    
#     return " ".join(translated_chunks)

# # Apply translation conditionally
# def apply_translation(row):
#     if row["language"] == "en":
#         return row["tokenized_sentences"]
#     else:
#         return translate_sentences(row["tokenized_sentences"])

# # Assuming data_1 is a Pandas DataFrame
# data_1["translated_text_2"] = data_1.apply(apply_translation, axis=1)


In [30]:
snippet=data_1.iloc[:10,:]

In [31]:
from deep_translator import GoogleTranslator
import pandas as pd
import ast  # To safely convert string representation of lists


# Function to split long texts into chunks
def chunk_text(text, chunk_size=4000):
    # Split the text into smaller chunks
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

# Function to translate tokenized sentences
def translate_sentences(tokenized_list, src_lang="auto", target_lang="en"):
   
        
        # Split text into smaller chunks
        chunks = chunk_text(tokenized_list)
        
        # Translate each chunk separately
        translated_chunks = [
            GoogleTranslator(source=src_lang, target=target_lang).translate(chunk) for chunk in chunks
        ]
        
        # Recombine the translated chunks into one string
        return " ".join(translated_chunks)
    


# Apply translation to tokenized sentences
snippet["translated_text"] = snippet["normalized_text"].apply(translate_sentences)

snippet.head(4)


RequestError: Request exception can happen due to an api connection error. Please check your connection and try again

In [ ]:
data_1.to_excel('test_3_transaltions.xlsx')

sources:

https://github.com/jfilter/clean-text/blob/main/README.md
